In [1]:
# !pip install natsort

In [2]:
import torch

torch.cuda.is_available()

True

In [3]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Laptop GPU'

In [4]:
torch.cuda.device_count()

1

# 데이터셋 선택 및 하이퍼파라미터 설정
▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼▼

In [5]:
import time
import natsort
import os

folder_list = os.listdir("./data/")
# .ipynb_checkpoints 폴더와 숨김 폴더 제외
item_list = [f for f in folder_list if not f.startswith('.')]
item_list = natsort.natsorted(item_list)

print("다음 데이터셋들이 학습됩니다 : ", item_list)

다음 데이터셋들이 학습됩니다 :  ['bottle', 'cable', 'capsule', 'carpet', 'grid', 'hazelnut', 'leather', 'metal_nut', 'pill', 'screw', 'tile', 'toothbrush', 'transistor', 'wood', 'zipper']


In [6]:
epochs = 50  # 200에서 50으로 줄임
batch_size = 8  # 16에서 32로 늘림 (GPU 메모리가 충분하면 64도 가능)

learning_rate = 0.005
image_size = 256

print(f"조정된 설정: epochs={epochs}, batch_size={batch_size}")
print(f"예상 학습 시간: 약 {15 * epochs * 0.5 / 60:.1f}시간")

조정된 설정: epochs=50, batch_size=8
예상 학습 시간: 약 6.2시간


▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲

# Main.py

In [7]:
import torch
from dataset import get_data_transforms
from torchvision.datasets import ImageFolder
import numpy as np
import random
import os
from torch.utils.data import DataLoader
# ResNet-50 사용
from resnet import resnet50, wide_resnet50_2
from de_resnet import de_resnet50, de_wide_resnet50_2
from dataset import RD_Dataset
import torch.backends.cudnn as cudnn
import argparse
from torch.nn import functional as F

In [8]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [9]:
def loss_fucntion(a, b):
    #mse_loss = torch.nn.MSELoss()
    cos_loss = torch.nn.CosineSimilarity()
    loss = 0
    for item in range(len(a)):
        #print(a[item].shape)
        #print(b[item].shape)
        #loss += 0.1*mse_loss(a[item], b[item])
        loss += torch.mean(1-cos_loss(a[item].view(a[item].shape[0],-1),
                                      b[item].view(b[item].shape[0],-1)))
    return loss

In [10]:
def loss_concat(a, b):
    mse_loss = torch.nn.MSELoss()
    cos_loss = torch.nn.CosineSimilarity()
    loss = 0
    a_map = []
    b_map = []
    size = a[0].shape[-1]
    for item in range(len(a)):
        #loss += mse_loss(a[item], b[item])
        a_map.append(F.interpolate(a[item], size=size, mode='bilinear', align_corners=True))
        b_map.append(F.interpolate(b[item], size=size, mode='bilinear', align_corners=True))
    a_map = torch.cat(a_map,1)
    b_map = torch.cat(b_map,1)
    loss += torch.mean(1-cos_loss(a_map,b_map))
    return loss

In [11]:
def train(_class_):
    print(_class_)
        
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(device)

    data_transform = get_data_transforms(image_size, image_size)
    
    train_path = './data/' + _class_ + '/train'
    ckp_path = './checkpoints/' + 'res50_'+_class_+'.pth'  # ResNet-50 checkpoint
    
    train_data = ImageFolder(root=train_path, transform=data_transform)
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

    # ResNet-50 사용 (일반 ResNet-50)
    encoder, bn = resnet50(pretrained=True)
    encoder = encoder.to(device)
    bn = bn.to(device)
    encoder.eval()
    decoder = de_resnet50(pretrained=False)
    decoder = decoder.to(device)

    optimizer = torch.optim.Adam(list(decoder.parameters())+list(bn.parameters()), lr=learning_rate, betas=(0.5,0.999))

    for epoch in range(epochs):
        start = time.time() 
        
        bn.train()
        decoder.train()
        loss_list = []
        for img, label in train_dataloader:
            img = img.to(device)
            inputs = encoder(img)
            outputs = decoder(bn(inputs))
            loss = loss_fucntion(inputs, outputs)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_list.append(loss.item())
        print('epoch [{}/{}], loss:{:.4f}'.format(epoch + 1, epochs, np.mean(loss_list)))
        print("time :",time.time() - start)
        
        if (epoch + 1) % 10 == 0:
            torch.save({'bn': bn.state_dict(),'decoder': decoder.state_dict()}, ckp_path)
            
    return loss

# 학습 시작

In [12]:
setup_seed(111)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [13]:
# bottle 데이터셋만 학습 (ResNet-50)
print("=" * 60)
print("Training ResNet-50 on bottle dataset")
print("=" * 60)

start_total = time.time()
train('bottle')
print(f"Total training time for bottle: {time.time() - start_total:.2f} seconds")
print("=" * 60)
print("Training completed! Checkpoint saved as: ./checkpoints/res50_bottle.pth")

Training ResNet-50 on bottle dataset
bottle
cuda


0.5%

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Aron/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


100.0%


epoch [1/50], loss:0.8988
time : 22.92002296447754
epoch [2/50], loss:0.3409
time : 10.89041018486023
epoch [3/50], loss:0.2190
time : 10.91915512084961
epoch [4/50], loss:0.1777
time : 10.808614253997803
epoch [5/50], loss:0.1570
time : 10.83558988571167
epoch [6/50], loss:0.1431
time : 12.181066751480103
epoch [7/50], loss:0.1373
time : 16.992446660995483
epoch [8/50], loss:0.1275
time : 17.754687070846558
epoch [9/50], loss:0.1230
time : 19.393676280975342
epoch [10/50], loss:0.1163
time : 18.612449407577515
epoch [11/50], loss:0.1126
time : 19.648245096206665
epoch [12/50], loss:0.1102
time : 17.848095178604126
epoch [13/50], loss:0.1070
time : 19.853597164154053
epoch [14/50], loss:0.1031
time : 19.495097398757935
epoch [15/50], loss:0.1008
time : 17.166810512542725
epoch [16/50], loss:0.0979
time : 15.53784441947937
epoch [17/50], loss:0.0971
time : 19.033114433288574
epoch [18/50], loss:0.0932
time : 18.27392601966858
epoch [19/50], loss:0.0908
time : 19.811684608459473
epoch [2